In [6]:
import pytorch_lightning as pl
import torch 
from pytorch_lightning.metrics.functional.classification import accuracy
from tqdm import tqdm
from src import DataModule, Resnet
import torchvision
import pandas as pd 
import numpy as np
from pathlib import Path
from skimage import io

Cargamos el modelo

In [7]:
checkpoint = torch.load('./resnet18-512-da-val_acc=0.87341.ckpt')
hparams = checkpoint['hyper_parameters']
hparams

{'lr': 0.0003,
 'optimizer': 'Adam',
 'batch_size': 128,
 'extra_data': 1,
 'subset': 0,
 'test_size': 0.2,
 'seed': 42,
 'backbone': 'resnet18',
 'size': 512,
 'train_trans': {'PadIfNeeded': {'min_height': 512,
   'min_width': 512,
   'border_mode': 0},
  'RandomResizedCrop': {'height': 512, 'width': 512},
  'HorizontalFlip': {},
  'VerticalFlip': {}},
 'val_trans': {'PadIfNeeded': {'min_height': 512,
   'min_width': 512,
   'border_mode': 0},
  'CenterCrop': {'height': 512, 'width': 512}},
 'precision': 16,
 'max_epochs': 50,
 'val_batches': 1.0}

In [8]:
model = Resnet.load_from_checkpoint(checkpoint_path='./resnet18-512-da-val_acc=0.87341.ckpt')
model.hparams

"backbone":    resnet18
"batch_size":  128
"extra_data":  1
"lr":          0.0003
"max_epochs":  50
"optimizer":   Adam
"precision":   16
"seed":        42
"size":        512
"subset":      0
"test_size":   0.2
"train_trans": {'PadIfNeeded': {'min_height': 512, 'min_width': 512, 'border_mode': 0}, 'RandomResizedCrop': {'height': 512, 'width': 512}, 'HorizontalFlip': {}, 'VerticalFlip': {}}
"val_batches": 1.0
"val_trans":   {'PadIfNeeded': {'min_height': 512, 'min_width': 512, 'border_mode': 0}, 'CenterCrop': {'height': 512, 'width': 512}}

In [9]:
size = 256
config = {
    'lr': 3e-4,
    'optimizer': 'Adam',
    'batch_size': 256,
    'max_epochs': 50,
    'precision': 16,
    'subset': 0,
    'test_size': 0.2,
    'seed': 42,
    'size': 256,
    'backbone': 'resnet18',
    'val_batches': 1.0,
    'extra_data': 0,
    'train_trans': {
        'CenterCrop': {
            'height': size, 
            'width': size
        }
    },
    'val_trans': {
        'CenterCrop': {
            'height': size, 
            'width': size
        }
    },
}

dm = DataModule(
    file = 'train_extra.csv' if config['extra_data'] else 'train_old.csv', 
    **config
)
dm.setup()

Training samples:  17117
Validation samples:  4280


In [10]:
 def evaluate(model, dl):   
    model.eval()
    model.cuda()
    acc = []
    with torch.no_grad():
        t = tqdm(dl)
        for x, y in t:
            x, y = x.cuda(), y.cuda()
            y_hat = model(x)
            acc.append(accuracy(y_hat, y).item())
            t.set_description(f"acc {np.mean(acc):.5f}")
            
evaluate(model, dm.val_dataloader())


acc 0.81140: 100%|██████████| 17/17 [00:54<00:00,  3.23s/it]


In [11]:
class Preprocess(torch.nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, x):
        x = x.float() / 255.
        x = x.permute(0, 3, 1, 2)
        return x 
    
class Postprocess(torch.nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, x):
        return torch.argmax(x, dim=1)

In [12]:
script = torch.jit.script(torch.nn.Sequential(
    Preprocess(),
    model.resnet.cpu(),
    Postprocess()
))
torch.jit.save(script, "model.pt")

In [13]:
 def evaluate2(model, dl):   
    model.eval()
    model.cuda()
    acc = []
    with torch.no_grad():
        t = tqdm(dl)
        for x, y in t:
            x, y = x.cuda(), y.cuda()
            # simulate test
            x *= 255. 
            x = x.permute(0, 2, 3, 1).long()
            #print(x.shape, x.dtype, x.max(), x.min())
            y_hat = model(x)
            acc.append(accuracy(y_hat, y).item())
            t.set_description(f"acc {np.mean(acc):.5f}")

In [14]:
loaded = torch.jit.load('model.pt')
evaluate2(loaded, dm.val_dataloader())


acc 0.81140: 100%|██████████| 17/17 [00:55<00:00,  3.28s/it]


In [ ]:
path = Path('./data/test_images')
images = os.listdir(path)
images_paths = [str(path/img) for img in images]
len(images)

In [ ]:
def predict(model, images, bs=32):   
    model.eval()
    model.cuda()
    preds = torch.tensor([]).cuda()
    batches = len(images) // bs + 1
    print(batches)
    with torch.no_grad():
        for b in tqdm(range(batches)):
            imgs = images[bs*b:bs*(b+1)]
            imgs = torch.from_numpy(np.array([io.imread(img) for img in imgs]))
            y_hat = model(imgs.cuda())
            preds = torch.cat([preds, y_hat])
    return preds.long().cpu().numpy()

In [ ]:
loaded = torch.jit.load('model.pt')
preds = predict(loaded, images_paths)
preds

In [ ]:
submission = pd.DataFrame({'image_id': images, 'label': preds })
submission

In [ ]:
submission.to_csv('submission.csv', index=False)